# FAST-UAV - Life Cycle Assessments of multirotor UAVs

[FAST-OAD](https://fast-oad.readthedocs.io) is a framework for performing rapid Overall Aircraft Design. The computational core of FAST-OAD is based on the  [OpenMDAO framework](https://openmdao.org/). <br>
FAST-UAV is the drone declination of FAST-OAD.

## 1. Setting up and analyzing the initial problem

To organize our work, we propose to use two user folders `data/` and `workdir/`. For instance, in `data/` we store a XML file which describes the [DJI Matrice 600 Pro](https://www.dji.com/matrice600-pro) multicopter. In `workdir/`, we store files generated or modified by FAST-UAV.

In [ ]:
import os.path as pth
import openmdao.api as om
import logging
import shutil
import fastoad.api as oad
from fastuav.cmd.lca import *
from time import time
from IPython.display import IFrame
import matplotlib.pyplot as plt
import brightway2 as bw
from fastuav.utils.postprocessing.analysis_and_plots import *
from fastuav.utils.postprocessing.lca import *

plt.rcParams["figure.figsize"] = 16, 8
plt.rcParams.update({"font.size": 13})

# Declare paths to folders and files
DATA_FOLDER_PATH = "./data"
WORK_FOLDER_PATH = "./workdir"
CONFIGURATION_FOLDER_PATH = pth.join(DATA_FOLDER_PATH, "configurations")
SOURCE_FOLDER_PATH = pth.join(DATA_FOLDER_PATH, "source_files")

CONFIGURATION_FILE = pth.join(CONFIGURATION_FOLDER_PATH, "multirotor_mdo_lca.yaml") # You may provide a different configuration file
SOURCE_FILE = pth.join(SOURCE_FOLDER_PATH, "problem_inputs_lca.xml") # You may provide a different source file

CONFIGURATION_FILE_LCA_ONLY = pth.join(CONFIGURATION_FOLDER_PATH, "multirotor_lca.yaml")

# For having log messages display on screen
logging.basicConfig(level=logging.INFO, format="%(levelname)-8s: %(message)s")

# For using all screen width
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:95% !important; }</style>"))

---------------
The YAML configuration file located in the data folder defines the design problem, i.e. the model, the problem driver and the optimization problem definition.<br>
A useful feature is the [N2 diagram](http://openmdao.org/twodocs/versions/latest/basic_guide/make_n2.html) visualization available in OpenMDAO to see the structure of the model:

In [ ]:
N2_FILE = pth.join(WORK_FOLDER_PATH, "n2.html")
oad.write_n2(CONFIGURATION_FILE, N2_FILE, overwrite=True)
IFrame(src=N2_FILE, width="100%", height="500px")

In [ ]:
#oad.generate_configuration_file(
#    CONFIGURATION_FILE_LCA_ONLY, overwrite=True, distribution_name="fastuav", sample_file_name="multirotor_lca.yaml"
#)
LCA_FILE = pth.join(WORK_FOLDER_PATH, "LCA_processes.html")
net = graph_activities(CONFIGURATION_FILE)
net.show(LCA_FILE)

In the configuration file, we have specified an input file name 'problem_inputs.xml'. We can ask FAST-UAV to generate the inputs of the model with the reference parameters from 'problem_inputs_DJI_M600.xml' as default values:

In [ ]:
oad.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

You can now checkout the generated [input file](./workdir/problem_inputs.xml). The values in this file can be modified by the user and will be considered by FAST-UAV when executing a computational process.<br>
The `variable-viewer` provides a way to inspect and modify the content of the XML file. The dropdown lists above the table allow to filter the displayed variable.

In [ ]:
INPUT_FILE = pth.join(WORK_FOLDER_PATH, "problem_inputs.xml")
oad.variable_viewer(INPUT_FILE)

## 2. Running an MDO

You can now run an optimization problem. The last part of the configuration file .yaml is where this optimization problem is defined:

```yaml
optimization:
  design_variables:
    - name: data:weights:mtow:k # over estimation coefficient on the load mass
      upper: 40.0
      lower: 1.0
  constraints:
    - name: data:weights:mtow:guess:constraint # mass consistency
      lower: 0.0
  objective:
    - name: data:weights:mtow
      scaler: 1e-1
```

In [ ]:
eval_problem = oad.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

In [ ]:
optim_problem = oad.optimize_problem(CONFIGURATION_FILE, overwrite=True)

In [ ]:
#import lca_algebraic as agb
#from lcav.io.configuration import LCAProblemConfigurator
#import sympy as sym

#_, model, methods = LCAProblemConfigurator('./data/lca/lca_model.yaml').generate()

#agb.lca.compute_impacts(model, methods, axis='phase')
#parameters = agb.all_params().values()

#lambdas = agb.lca._preMultiLCAAlgebric(model, methods)

# Compile expressions for partial derivatives of impacts
#partial_lambdas_dict = {method: [lambd.expr.diff(param)]}
#partial_lambdas_dict = {param.name: [agb.lca.LambdaWithParamNames(lambd.expr.replace(sym.ceiling, lambda x: x).diff(param)) for lambd in lambdas] for param in parameters} 

Let's save these results:

In [ ]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, "problem_outputs.xml")
MDO_OUTPUT_FILE = pth.join(DATA_FOLDER_PATH, 'problem_outputs_lca_mdo.xml')
shutil.copy(OUTPUT_FILE, MDO_OUTPUT_FILE)

The `optimizer_viewer` offers a convenient summary of the optimization result. If design variables or constraints have active bounds they are yellow whereas they are red if they are violated.

In [ ]:
oad.optimization_viewer(CONFIGURATION_FILE)

You can use the `VariableViewer` tool to see the optimization results for all variables of the system by loading the .xml output file:

In [ ]:
oad.variable_viewer(MDO_OUTPUT_FILE)

## 3. Running the LCA on an existing design
For instance, you may want to change the UAV's lifetime or the electricity mix to study the effect of these parameters on the environmental impacts.

First, we load the configuration file containing only the LCA module:

And we generate the input file for the LCA based on the previous output file:

In [ ]:
MDO_OUTPUT_FILE = pth.join(DATA_FOLDER_PATH, 'problem_outputs_lca_mdo.xml')
oad.generate_inputs(CONFIGURATION_FILE_LCA_ONLY, MDO_OUTPUT_FILE, overwrite=True)

Let's check the inputs of the LCA module. You can fill in the new values for the LCA parameters.

In [ ]:
INPUT_FILE = pth.join(WORK_FOLDER_PATH, "problem_inputs.xml")
oad.variable_viewer(INPUT_FILE)

Finally, we run the LCA module with the updated parameters.

In [ ]:
eval_problem = oad.evaluate_problem(CONFIGURATION_FILE_LCA_ONLY, overwrite=True)

In [ ]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, "problem_outputs.xml")
LCA_OUTPUT_FILE = pth.join(DATA_FOLDER_PATH, 'problem_outputs_lca.xml')
shutil.copy(OUTPUT_FILE, LCA_OUTPUT_FILE)

In [ ]:
oad.variable_viewer(LCA_OUTPUT_FILE)

## 4. Analysis and plots

You can now use postprocessing plots to visualize the results of the MDO.

### 4.1 - Geometry and mass breakdown

In [ ]:
fig = multirotor_geometry_plot(MDO_OUTPUT_FILE, name="Drone MDO")
fig.show()
#plotly.io.write_image(fig, 'output_file.pdf', format='pdf')

In [ ]:
fig = mass_breakdown_sun_plot_drone(MDO_OUTPUT_FILE)
fig.show()
#plotly.io.write_image(fig, 'output_file.pdf', format='pdf')

### 4.2 - LCA results

In [ ]:
# Weighted single score
fig = lca_plot(OUTPUT_FILE, result_step = 'aggregation', filter_option = 'default', filter_level = 1)
fig.update_layout(
    title=None,
    margin=dict(l=10, r=10, t=0, b=0),
    width=350,
    height=280
)
colors = [px.colors.qualitative.Plotly[0], px.colors.qualitative.Plotly[9]]
fig.update_traces(marker=dict(colors=colors))
fig.update_traces(texttemplate='%{label} %{percent:.0%}')
fig.show()
plotly.io.write_image(fig, 'output_file.pdf', format='pdf')

In [ ]:
# Weighted single score
fig = lca_plot(OUTPUT_FILE, result_step = 'aggregation', filter_option = 'exact', filter_level = 2)
fig.update_layout(
    title=None,
    margin=dict(l=10, r=10, t=0, b=0),
    width=350,
    height=280
)
colors = [px.colors.qualitative.Plotly[i] for i in range(1,10)]
fig.update_traces(marker=dict(colors=colors))
fig.update_traces(texttemplate='%{label} %{percent:.0%}')
fig.show()
#plotly.io.write_image(fig, 'output_file.pdf', format='pdf')

In [ ]:
# Normalized and weighted scores for each impact category
fig = lca_plot(OUTPUT_FILE, result_step = 'weighting', filter_option = 'default', filter_level = 2, percent = False)
fig.update_layout(title=None, width=1000, height=800, paper_bgcolor='rgba(0,0,0,0)',  plot_bgcolor='rgba(0,0,0,0)', font=dict(size=14), legend_traceorder="reversed")
fig.update_xaxes(showline=True, linewidth=0.5, linecolor='black', tickangle=90)
fig.update_yaxes(title='Points', showline=True, linewidth=1, linecolor='black', gridcolor='grey', gridwidth=0.05)

for idx in range(len(fig.data)):
    fig.data[idx].x = [s.split("<br>", 2)[1] for s in fig.data[idx].x]
    fig.data[idx].marker.line.width = 0
    
#y_errors = [0.0010183, 0.0010032, 0.0521486, 0.0014123, 0.0066393, 0.0002067, 0.0003722, 0.0518777, 0.0006717858, 0.0090101, 0.0000846, 0.0461122, 0.0000046, 0.0014748, 0.0004816, 0.0014202]
#fig.data[-1].error_y=dict(type="data", array=y_errors)

fig.show()
plotly.io.write_image(fig, 'output_file.pdf', format='pdf')

In [ ]:
# Processes contributions in each impact category
fig = lca_plot(OUTPUT_FILE, result_step = 'characterization', filter_option = 'default', filter_level = 2, percent = True)

fig.update_layout(title=None, width=1000, height=920, paper_bgcolor='rgba(0,0,0,0)',  plot_bgcolor='rgba(0,0,0,0)', font=dict(size=14), legend_traceorder="reversed")
fig.update_xaxes(showline=True, linewidth=0.5, linecolor='black', tickangle=90)
fig.update_yaxes(title='Relative share', showline=True, linewidth=1, linecolor='black', gridcolor='grey', gridwidth=0.05)

for idx in range(len(fig.data)):
    fig.data[idx].x = [(s.split("<br>")[1] + " " + s.split("<br>")[3]).replace("/FU", "").replace("dimensionless", "soil quality index") for s in fig.data[idx].x]
    fig.data[idx].marker.line.width = 0
    
fig.show()
plotly.io.write_image(fig, 'output_file.pdf', format='pdf')

#### **Splitting "operation" contribution into terms related to the components' masses and efficiencies.**

This analysis of the specific component contributions is only valid for cruise conditions.

In [ ]:
figs = lca_specific_contributions(OUTPUT_FILE, result_step = 'aggregation')
for fig in figs:
    fig.show()

In [ ]:
# Bar plot export
fig = lca_specific_contributions(OUTPUT_FILE, result_step = 'aggregation')[0]

for idx in range(len(fig.data)):
    if fig.data[idx].legendgroup == 'bar plot':# remove 'payload' and 'misc' from bar plot
        fig.data[idx].y = [fig.data[idx].y[i] for i in range(len(fig.data[idx].x)) if fig.data[idx].x[i] not in ['payload', 'misc']] 
        fig.data[idx].x = [fig.data[idx].x[i] for i in range(len(fig.data[idx].x)) if fig.data[idx].x[i] not in ['payload', 'misc']]
fig.data = [fig.data[idx] for idx in range(len(fig.data)) if fig.data[idx].name not in ['payload', 'misc']]  # remove from ternary plot

import matplotlib.colors
#colors = [px.colors.qualitative.T10[2], px.colors.qualitative.D3[7], px.colors.qualitative.Plotly[9]]
colors = [px.colors.qualitative.Plotly[i] for i in range(1, 6)]
opacities = [0.2, 0.5, 1.0]
patterns = ["x", ".", "/"]
fig_bar = go.Figure(data=[fig.data[idx] for idx in range(len(fig.data)) if fig.data[idx].legendgroup == 'bar plot'])
fig_bar.update_layout(title=None, width=600, height=400, 
                      paper_bgcolor='rgba(0,0,0,0)',  plot_bgcolor='rgba(0,0,0,0)', 
                      font=dict(size=14), margin=dict(l=10, r=10, t=5, b=5), 
                      barmode='stack', xaxis={'categoryorder': 'total descending'},
                      showlegend=True
                     )
for idx in range(len(fig_bar.data)):
    fig_bar.data[idx].marker.color = ['rgba' + str(matplotlib.colors.to_rgb(color) + (opacities[idx],)).replace('1.0', '0.99') for color in colors]
    fig_bar.data[idx].marker.line.width = 0.8
    fig_bar.data[idx].marker.line.color = 'black'
    fig_bar.data[idx].marker.pattern.shape = [patterns[idx] for i in range(1,6)]
fig_bar.update_xaxes(showline=True, linewidth=0.5, linecolor='black', tickangle= 45)
fig_bar.update_yaxes(title='Single score (points)', showline=True, linewidth=1, linecolor='black', gridcolor='grey', gridwidth=0.05, range=[0.0, 0.05])
fig_bar.update_traces(showlegend=False).add_traces(
    [
        go.Bar(name=key, x=[None], marker_color=val[0], showlegend=True, marker_pattern_shape=val[1])
        for key, val in {"mass": ['rgba(.0, .0, .0, 0.05)', "x"],
                         "efficiency": ['rgba(.0, .0, .0, 0.12)', "."],
                         "production": ['rgba(.0, .0, .0, 0.3)', "/"],
                        }.items()
    ]
)

fig_bar.show()
plotly.io.write_image(fig_bar, 'output_file.pdf', format='pdf', scale=5)

In [ ]:
# Ternary plot export
fig_ternary = go.Figure(data=[fig.data[idx] for idx in range(len(fig.data)) if fig.data[idx].legendgroup == 'ternary plot'])
for idx in range(len(fig_ternary.data)):
    fig_ternary.data[idx].marker.size *= 12
fig_ternary.update_layout(title=None, width=600, height=400, paper_bgcolor='rgba(0,0,0,0)',  plot_bgcolor='rgba(0,0,0,0)', font=dict(size=14), legend=dict(itemsizing='constant'), margin=dict(l=50, r=50, t=50, b=50))
fig_ternary.update_ternaries(bgcolor='rgba(0,0,0,0)', 
                             aaxis_linecolor='black',
                             baxis_linecolor='black',
                             caxis_linecolor='black',
                             aaxis_gridcolor='lightgrey', 
                             baxis_gridcolor='lightgrey', 
                             caxis_gridcolor='lightgrey', 
                             aaxis_gridwidth=0.01,
                             baxis_gridwidth=0.01,
                             caxis_gridwidth=0.01,
                             sum=1,
                             aaxis_title= 'Mass', 
                             baxis_title= 'Efficiency', 
                             caxis_title= 'Production',
                             aaxis_ticks='outside',
                             baxis_ticks='outside',
                             caxis_ticks='outside',
                            )
colors = [px.colors.qualitative.Plotly[i] for i in range(1, 10)]
for idx in range(len(fig_ternary.data)):
    fig_ternary.data[idx].marker.color = colors[idx]
    fig_ternary.data[idx].marker.line.width = 0.8
    fig_ternary.data[idx].marker.line.color = 'black'

fig_ternary.show()
plotly.io.write_image(fig_ternary, 'output_file.pdf', format='pdf')

In [ ]:
figs = lca_specific_contributions(OUTPUT_FILE, result_step = 'characterization')
for fig in figs:
    fig.show()

### 4.3 - Advanced LCA studies

#### Preliminary

In [ ]:
# get all activities created in LCA module
activities = get_lca_activities()
activities

In [ ]:
# select top-level activity (model)
model = get_lca_main_activity()
model

In [ ]:
# get a particular activity
from fastuav.constants import LCA_USER_DB
act = lcalg.getActByCode(LCA_USER_DB, 'production')

In [ ]:
# Another way of visualizing interactions
recursive_activities(model)

In [ ]:
# list available parameters
list_lca_parameters()  # returns HTML table
# lcalg.params._variable_params()  # returns dictionnary of parameters

#### 1 - Uncertainty analysis with Monte Carlo

The vanilla Monte Carlo from brightway2 is used to calculate the uncertainty on each impact. Only uncertainties from EcoInvent datasets are taken into account. No uncertainty on characterization factors is provided by standard methods. Finally, values from algebraic parameters (e.g., components masses) are frozen.

In [ ]:
from lca_algebraic import multiLCA
res = multiLCA(model, methods, **parameters)

In [ ]:
res

In [ ]:
from fastoad.io import VariableIO
from fastuav.constants import LCA_PARAM_KEY, LCA_DEFAULT_METHOD
import time

# Select model
model = get_lca_main_activity()  # top-level model

# Get parameters values from problem outputs
variables = VariableIO(OUTPUT_FILE).read()
param_names = [p for p in variables.names() if p.startswith(LCA_PARAM_KEY)]
parameters = {}
for p in param_names:
    parameters[p.replace(LCA_PARAM_KEY, "")] = variables[p].value[0]

# Run Monte Carlo
methods = [eval(m) for m in LCA_DEFAULT_METHOD]
res = lca_monte_carlo(
    model, # the model
    methods, # impacts to assess 

    # Number of Monte Carlo runs
    n_runs=1000, 
    
    # Whether uncertainty on characterization factors is taken into account or not
    cfs_uncertainty = False,

    # Parameters of the model
    **parameters
)
res.to_csv("./workdir/lca_monte_carlo.csv")

In [ ]:
from fastoad.io import VariableIO
from fastuav.constants import LCA_PARAM_KEY, LCA_DEFAULT_METHOD
import time

# Select model
model = get_lca_main_activity()  # top-level model

# Get parameters values from problem outputs
variables = VariableIO(OUTPUT_FILE).read()
param_names = [p for p in variables.names() if p.startswith(LCA_PARAM_KEY)]
parameters = {}
for p in param_names:
    parameters[p.replace(LCA_PARAM_KEY, "")] = variables[p].value[0]

# Run Monte Carlo
methods = [eval(m) for m in LCA_DEFAULT_METHOD]

In [ ]:
res.describe()

In [ ]:
df = pd.read_csv("./workdir/lca_monte_carlo.csv", header=[0,1,2], index_col=0)

In [ ]:
df.quantile([0.025, 0.975])

In [ ]:
df.median()

In [ ]:
df.describe()

In [ ]:
# Plot distributions
fig = make_subplots(rows=4, cols=4, subplot_titles=[s[1].replace(':', '<br>') for s in df.columns.tolist()], horizontal_spacing = 0.08, vertical_spacing = 0.13)
axes_units=[r"$mol H^+_{eq}$", r"$kgCO_{2eq}$", '$CTU_e$', "$MJ$", r"$kgP_{eq}$", r"$kgN_{eq}$", r"$mol N_{eq}$", "$CTUh$", "$CTUh$", r"$kBq U235_{eq}$", r"$\text{soil quality index}$", r"$kg Sb_{eq}$", r"$kg CFC-11_{eq}$", r"$\text{disease incidence}$", r"$kg NMVOC_{eq}$", r"$m^3 \text{ world eq. deprived}$"]
deterministic_values = [4.741456481, 502.6635915, 3756.514583, 9683.489122, 0.434746217, 0.608561593, 5.591831523, 1.53206E-06, 2.67813E-05, 214.0044574, 2328.077792, 0.029507826, 1.39928E-05, 2.56764E-05, 1.759051648, 630.7958567]

for i in range(len(df.columns)): 
    fig.add_trace(
        go.Histogram(x=df.iloc[:, i], histnorm='probability'),
        row=i // 4 + 1, 
        col=i % 4 + 1,
    )
    fig.add_vline(x=deterministic_values[i], line_dash = 'dash',
                  row=i // 4 + 1, 
                  col=i % 4 + 1,
                 )
    fig.update_xaxes(title_text=axes_units[i], row=i // 4 + 1, col=i % 4 + 1, titlefont=dict(size=14))
    
for i in range(4):
    for j in range(3):
        fig.update_yaxes(showticklabels=True, matches=f'y{4*i+j+2}', row=i+1, col=j+1)
    
fig.update_layout(title=None, width=1200, height=1200, paper_bgcolor='rgba(0,0,0,0)',  plot_bgcolor='rgba(0,0,0,0)', font=dict(size=14), legend_traceorder="reversed", showlegend=False)
fig.update_xaxes(showline=True, linewidth=0.5, linecolor='black')
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', titlefont=dict(size=14))
fig.update_yaxes(title='probability', col=1)
fig.show()
plotly.io.write_image(fig, 'output_file.pdf', format='pdf')

#### 2 - Impacts variation with number of cycles

A design of experiments is achieved on the LCA parameter `n_cycles_uav` to plot the variations of the environmental impacts with the UAV's lifespan.

**Warning:** here no iteration on the design is made. DoE on the design plus the LCA can be made by hand or with a DoE on the whole FAST-UAV model.

In [ ]:
import lca_algebraic as lcalg
from fastoad.io import VariableIO
from fastuav.constants import LCA_PARAM_KEY, LCA_USER_DB, LCA_DEFAULT_METHOD

# Select activities/model to explore
model = get_lca_main_activity()
operation = lcalg.getActByCode(LCA_USER_DB, 'operation')
production = lcalg.getActByCode(LCA_USER_DB, 'production')
batteries = lcalg.getActByCode(LCA_USER_DB, 'batteries')
propellers = lcalg.getActByCode(LCA_USER_DB, 'propellers')
motors = lcalg.getActByCode(LCA_USER_DB, 'motors')
controllers = lcalg.getActByCode(LCA_USER_DB, 'controllers')
airframe = lcalg.getActByCode(LCA_USER_DB, 'airframe')

In [ ]:
# Get parameters values from problem outputs
variables = VariableIO(OUTPUT_FILE).read()
param_names = [p for p in variables.names() if p.startswith(LCA_PARAM_KEY)]
parameters = {}
for p in param_names:
    parameters[p.replace(LCA_PARAM_KEY, "")] = variables[p].value[0]
parameters

In [ ]:
# Set non-float parameters
parameters['elec_switch_param'] = "eu"
parameters['battery_type'] = "si_nmc_811" # "si_nmc_811" # "lfp"

# Modify parameter of interest
parameters['n_cycles_uav'] = list(np.linspace(1.0, 10000.0, 100000)) # list(np.geomspace(100.0, 5000, 10000))

# Choose lcia method
#method = [('ReCiPe 2016 v1.03, midpoint (E) no LT', 'climate change no LT', 'global warming potential (GWP1000) no LT')]
methods = [eval(m) for m in LCA_DEFAULT_METHOD]

# activities and sub-activities to evaluate
activities = [operation, batteries, propellers, controllers, airframe, motors]

# Run LCA. The DoE is automatically performed.
dict_df = {}
for act in activities:
    res = lcalg.multiLCAAlgebric(
        model, # The model 
        methods, # Impacts
        
        extract_activities=[act],
        
        # Parameters of the model
        **parameters
    )
    res.index = parameters['n_cycles_uav']
    dict_df[act.as_dict()['name']] = res

In [ ]:
# Compute single weighted score (very ugly way of doing that...)
for act in activities:
    act_name = act.as_dict()['name']
    dict_df[act_name]['single_score'] = dict_df[act_name]['EF_v3_1:acidification:accumulated_exceedance_AE[mol H+-Eq]'] / 55.569541230602 * 0.062 \
    + dict_df[act_name]['EF_v3_1:climate_change:global_warming_potential_GWP100[kg CO2-Eq]'] / 8095.52506394406 * 0.2106 \
    + dict_df[act_name]['EF_v3_1:ecotoxicity-_freshwater:comparative_toxic_unit_for_ecosystems_CTUe[CTUe]'] / 42683.1618655979 * 0.0192 \
    + dict_df[act_name]['EF_v3_1:energy_resources-_non-renewable:abiotic_depletion_potential_ADP-_fossil_fuels[MJ, net calorific value]'] / 65004.2596640167 * 0.0832 \
    + dict_df[act_name]['EF_v3_1:eutrophication-_freshwater:fraction_of_nutrients_reaching_freshwater_end_compartment_P[kg P-Eq]'] / 1.60685212828813 * 0.0280 \
    + dict_df[act_name]['EF_v3_1:eutrophication-_marine:fraction_of_nutrients_reaching_marine_end_compartment_N[kg N-Eq]'] / 19.5451815519191 * 0.0296 \
    + dict_df[act_name]['EF_v3_1:eutrophication-_terrestrial:accumulated_exceedance_AE[mol N-Eq]'] / 176.754999788942 * 0.0371 \
    + dict_df[act_name]['EF_v3_1:human_toxicity-_carcinogenic:comparative_toxic_unit_for_human_CTUh[CTUh]'] / 0.000016899507395756 * 0.0213 \
    + dict_df[act_name]['EF_v3_1:human_toxicity-_non-carcinogenic:comparative_toxic_unit_for_human_CTUh[CTUh]'] / 0.000229659215899932 * 0.0184 \
    + dict_df[act_name]['EF_v3_1:ionising_radiation-_human_health:human_exposure_efficiency_relative_to_u235[kBq U235-Eq]'] / 4220.15981253385 * 0.0501 \
    + dict_df[act_name]['EF_v3_1:land_use:soil_quality_index[dimensionless]'] / 819498.182923031 * 0.0794 \
    + dict_df[act_name]['EF_v3_1:material_resources-_metals_minerals:abiotic_depletion_potential_ADP-_elements_ultimate_reserves[kg Sb-Eq]'] / 0.0636402782259556 * 0.0755 \
    + dict_df[act_name]['EF_v3_1:ozone_depletion:ozone_depletion_potential_ODP[kg CFC-11-Eq]'] / 0.0536479905672634 * 0.0631 \
    + dict_df[act_name]['EF_v3_1:particulate_matter_formation:impact_on_human_health[disease incidence]'] / 0.000595386937135986 * 0.0896 \
    + dict_df[act_name]['EF_v3_1:photochemical_oxidant_formation-_human_health:tropospheric_ozone_concentration_increase[kg NMVOC-Eq]'] / 40.6013974614544 * 0.0478 \
    + dict_df[act_name]['EF_v3_1:water_use:user_deprivation_potential_deprivation-weighted_water_consumption[m3 world eq. deprived]'] / 11468.7086407597 * 0.0851

In [ ]:
# Save dataframes to csv files
for act in activities:
    act_name = act.as_dict()['name']
    df_act = dict_df[act_name]
    df_act.to_csv("./workdir/df_cycles_" + act_name + ".csv")

In [ ]:
# Create and save dataframe containing only the single scores for each activity
df_merged = pd.concat([dict_df[act.as_dict()['name']]["single_score"] for act in activities], axis=1, keys=[act.as_dict()['name'] for act in activities])
df_merged.to_csv("./workdir/df_cycles_single_scores.csv")

In [ ]:
# Plot the evolution of the single score with the number of cycles
df_merged = pd.read_csv("./workdir/df_cycles_single_scores.csv")
activities_names = ['airframe', 'controllers', 'motors', 'propellers', 'operation', 'batteries']
colors = [px.colors.qualitative.Plotly[1], px.colors.qualitative.Plotly[3], px.colors.qualitative.Plotly[4], px.colors.qualitative.Plotly[5], px.colors.qualitative.Plotly[0], px.colors.qualitative.Plotly[2]]
#fig = px.area(df_merged, x='n_cycles', y=activities_names, groupnorm='percent')
fig = px.area(df_merged, x='n_cycles', y=activities_names, groupnorm=None, color_discrete_sequence=colors)
fig.update_layout(title=None, width=600, height=360, paper_bgcolor='rgba(0,0,0,0)',  plot_bgcolor='rgba(0,0,0,0)', font=dict(size=14), legend_title="", legend_traceorder="reversed", )
fig.update_xaxes(title='Number of cycles (-)', showline=True, linewidth=1, linecolor='black', gridcolor='lightgrey', gridwidth=0.01, tickformat=".f")#, range=[0,2600])
fig.update_yaxes(title='Single score (points)', showline=True, linewidth=1, linecolor='black', gridcolor='lightgrey', gridwidth=0.01)#, range=[0,0.18])
#fig.for_each_trace(lambda trace: trace.update(fillcolor = trace.line.color, marker_line_color = 'black', marker_line_width = 1.0))
fig.show()
plotly.io.write_image(fig, 'output_file.pdf', format='pdf')

In [ ]:
# With log scale (not used)
#df_merged = pd.read_csv("./workdir/df_cycles_single_scores.csv")
#activities_names = ['propellers', 'controllers', 'airframe', 'motors', 'batteries', 'operation']
#fig = px.area(df_merged, x='n_cycles', y=activities_names, groupnorm=None)
#fig.update_layout(title=None, width=1000, height=600, paper_bgcolor='rgba(0,0,0,0)',  plot_bgcolor='rgba(0,0,0,0)', font=dict(size=14), legend_title="", legend_traceorder="reversed")
#fig.update_xaxes(title='Number of cycles (-)', showline=True, linewidth=1, linecolor='black', gridcolor='lightgrey', gridwidth=0.01, tickformat=".f")
#tickvals = np.concatenate((np.arange(0.001, 0.01, 0.001),
#                           np.arange(0.01, 0.1, 0.01),
#                           np.arange(0.1, 1.1, 0.1)))
#ticktext = [str(val) if val in [0.001, 0.01, 0.1, 1] else '' for val in tickvals]
#fig.update_yaxes(title='Single score (points)', showline=True, linewidth=1, linecolor='black', gridcolor='lightgrey', gridwidth=0.01, 
#                 type='log',
#                 tickmode="array",
#                 tickvals=tickvals,
#                 ticktext=ticktext
#                )  #tickformat = ".1r"
#fig.show()
#plotly.io.write_image(fig, 'output_file.pdf', format='pdf')

#### 3 - Variations of components contributions with number of cycles

Here we carry a (simple) design of experiments on the LCA module of FAST-UAV. Although it is more time consuming compared to using lca_algebraic's bundled DoE, it enables to get access to the outputs returned by FAST-UAV. Here, we want to have access to the components' contributions as calculated by the LCA postprocessing module of FAST-UAV.

In [ ]:
from fastuav.utils.postprocessing.sensitivity_analysis.sensitivity_analysis import doe_fast

# Inputs
x_dict = {
    "lca:parameters:n_cycles_uav": np.linspace(1.0, 10000.0, 1000),
         }

# Outputs
y_list = [
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:airframe',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:airframe:efficiency',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:airframe:mass',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:airframe:production',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:batteries',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:batteries:efficiency',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:batteries:mass',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:batteries:production',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:controllers',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:controllers:efficiency',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:controllers:mass',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:controllers:production',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:motors',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:motors:efficiency',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:motors:mass',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:motors:production',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:payload',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:payload:mass',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:propellers',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:propellers:efficiency',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:propellers:mass',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:propellers:production',
]

# Results of DoE
df = doe_fast("list", x_dict, y_list, CONFIGURATION_FILE_LCA_ONLY)  # Run DoE on LCA model
df.to_csv('./workdir/df_components_contributions.csv')

In [ ]:
import matplotlib

df.columns = df.columns.str.replace('lca:postprocessing:aggregation:weighted_single_score:', '')
component_name = 'batteries'

color = px.colors.qualitative.Plotly[2]
opacities = [0.2, 0.5, 1.0]
patterns = ['x', '.', '/']
fig = px.area(df, x='lca:parameters:n_cycles_uav', 
              y=['model_per_FU:' + component_name + ':mass', 
                 'model_per_FU:' + component_name + ':efficiency',
                 'model_per_FU:' + component_name + ':production'
                ], 
              groupnorm='fraction',
              color_discrete_sequence=[color])
newnames = {'model_per_FU:' + component_name + ':efficiency':'efficiency', 'model_per_FU:' + component_name + ':mass':'mass', 'model_per_FU:' + component_name + ':production':'production'}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
for idx in range(len(fig.data)):
    fig.data[idx].fillcolor = 'rgba' + str(matplotlib.colors.to_rgb(color) + (opacities[idx],)).replace('1.0', '0.99')
    fig.data[idx].line.width = 1.2
    fig.data[idx].line.color = 'black'
    fig.data[idx].fillpattern.shape = patterns[idx]
fig.add_annotation(x=9500, y=0.6,
            text="production",
            showarrow=True,
            ax=80,
            ay=0,
            arrowsize=1.5,
            arrowhead=1)
fig.add_annotation(x=9500, y=0.06,
            text="efficiency",
            showarrow=True,
            ax=80,
            ay=-40,
            arrowsize=1.5,
            arrowhead=1)
fig.add_annotation(x=9500, y=0.015,
            text="mass",
            showarrow=True,
            ax=65,
            ay=-10,
            arrowsize=1.5,
            arrowhead=1)
fig.update_layout(title=None, width=600, height=350, paper_bgcolor='rgba(0,0,0,0)',  plot_bgcolor='rgba(0,0,0,0)', font=dict(size=14), legend_title="", legend_traceorder="reversed", showlegend=False)
fig.update_xaxes(title='Number of cycles (-)', showline=True, linewidth=1, linecolor='black', tickformat=".f", showgrid=False, gridwidth=0.0, tickvals=[0, 2000, 4000, 6000, 8000, 10000], ticks="outside") #, range=[0,10000])
fig.update_yaxes(title='Single score', showline=True, linewidth=1, linecolor='black', tickformat='.0%', showgrid=False, gridwidth=0.0, range=[0,1.0])
fig.show()
plotly.io.write_image(fig, 'output_file.pdf', format='pdf', scale=5)

#### 4 - Optimization Paretos

In [ ]:
import os

# Create DataFrame to store points of Pareto
df = pd.DataFrame()

In [ ]:
# Fresh start
oad.generate_configuration_file(
    CONFIGURATION_FILE, overwrite=True, distribution_name="fastuav", sample_file_name="multirotor_mdo_lca.yaml"
)
oad.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)
INPUT_FILE = pth.join(WORK_FOLDER_PATH, "problem_inputs.xml")
datafile = oad.DataFile(INPUT_FILE)
#datafile.save()

In [ ]:
# MTOW minimization objective --> get minimum MTOW value (first extremum of Pareto)

# Declare optimization problem
conf = oad.FASTOADProblemConfigurator(CONFIGURATION_FILE)
prob_definition = conf.get_optimization_definition()
prob_definition['objective'] = {'data:weight:mtow': {
    'name': 'data:weight:mtow',
    'scaler': 0.1}}
conf.set_optimization_definition(prob_definition)
prob = conf.get_problem(read_inputs=True, auto_scaling=False)

# Attach recorder to the driver
if os.path.exists("cases.sql"):
    os.remove("cases.sql")
prob.driver.add_recorder(om.SqliteRecorder("cases.sql"))
prob.driver.recording_options["includes"] = ['*']  # include all variables from the problem

# Run problem
prob.setup()  # setup problem
prob.optim_failed = prob.run_driver()  # run optimization
prob.cleanup()

# Get results from recorded cases and store them
cr = om.CaseReader("cases.sql")
case = cr.get_case(-1)
values = {key:case.outputs[key] for key in case.outputs if case.outputs[key].shape == (1,)}  # get one-dimensional variables
df = pd.concat([df, pd.DataFrame(values)], ignore_index=True)  # store in dataframe

In [ ]:
# Energy minimization objective (second extremum of Pareto)

# Declare optimization problem
conf = oad.FASTOADProblemConfigurator(CONFIGURATION_FILE)
prob_definition = conf.get_optimization_definition()
prob_definition['objective'] = {'mission:operational:energy': {
    'name': 'mission:operational:energy',
    'scaler': 0.001}}
conf.set_optimization_definition(prob_definition)
prob = conf.get_problem(read_inputs=True, auto_scaling=False)
prob.driver = om.ScipyOptimizeDriver(tol=1e-4, optimizer='SLSQP', maxiter=30)

# Attach recorder to the driver
if os.path.exists("cases.sql"):
    os.remove("cases.sql")
prob.driver.add_recorder(om.SqliteRecorder("cases.sql"))
prob.driver.recording_options["includes"] = ['*'] 

# Run problem
prob.setup()  # setup problem
prob.optim_failed = prob.run_driver()  # run optimization
prob.cleanup()

# Get results from recorded cases and store them
cr = om.CaseReader("cases.sql")
case = cr.get_case(-1)
values = {key:case.outputs[key] for key in case.outputs if case.outputs[key].shape == (1,)}  # get one-dimensional variables
df = pd.concat([df, pd.DataFrame(values)], ignore_index=True)  # store in dataframe

In [ ]:
# LCA score minimization objective (intermediate extremum of Pareto)

# Declare optimization problem
conf = oad.FASTOADProblemConfigurator(CONFIGURATION_FILE)
prob_definition = conf.get_optimization_definition()
prob_definition['objective'] = {'lca:aggregation:weighted_single_score:model_per_FU': {
    'name': 'lca:aggregation:weighted_single_score:model_per_FU',
    'scaler': 10.0}}
conf.set_optimization_definition(prob_definition)
prob = conf.get_problem(read_inputs=True, auto_scaling=False)
prob.driver = om.ScipyOptimizeDriver(tol=1e-4, optimizer='SLSQP', maxiter=30)

# Attach recorder to the driver
if os.path.exists("cases.sql"):
    os.remove("cases.sql")
prob.driver.add_recorder(om.SqliteRecorder("cases.sql"))
prob.driver.recording_options["includes"] = ['*'] 

# Run problem
prob.setup()  # setup problem
prob.optim_failed = prob.run_driver()  # run optimization
prob.cleanup()

# Get results from recorded cases and store them
cr = om.CaseReader("cases.sql")
case = cr.get_case(-1)
values = {key:case.outputs[key] for key in case.outputs if case.outputs[key].shape == (1,)}  # get one-dimensional variables
df = pd.concat([df, pd.DataFrame(values)], ignore_index=True)  # store in dataframe

In [ ]:
# DoE between the two optimums to construct the Pareto front

# Setup
num_points = 20
mtow_array = np.linspace(min(df['data:weight:mtow']), max(df['data:weight:mtow']), num_points+2)[1:-1]  # don't need to re-run the two extreme cases

# Declare optimization problem
conf = oad.FASTOADProblemConfigurator(CONFIGURATION_FILE)
prob_definition = conf.get_optimization_definition()

#prob_definition['objective'] = {'lca:aggregation:weighted_single_score:model_per_FU': {
#    'name': 'lca:aggregation:weighted_single_score:model_per_FU',
#    'scaler': 10.0}}
prob_definition['objective'] = {'mission:operational:energy': {
    'name': 'mission:operational:energy',
    'scaler': 0.001}}
prob_definition['constraints']['data:weight:mtow:requirement:constraint'] = {
    'name': 'data:weight:mtow:requirement:constraint',
    'lower': 0.0}
conf.set_optimization_definition(prob_definition)
prob = conf.get_problem(read_inputs=True, auto_scaling=False)
prob.driver = om.ScipyOptimizeDriver(tol=1e-4, optimizer='SLSQP', maxiter=30)

for mtow in mtow_array:
    # Set mtow constraint
    datafile['data:weight:mtow:requirement'].val = mtow
    datafile.save()
    
    # Attach recorder to the driver
    if os.path.exists("cases.sql"):
        os.remove("cases.sql")
    prob.driver.add_recorder(om.SqliteRecorder("cases.sql"))
    prob.driver.recording_options["includes"] = ['*'] 
    
    # run problem
    prob.setup()  # setup problem
    prob.optim_failed = prob.run_driver()  # run optimization
    prob.cleanup()
    
    # Get results from recorded cases and store them
    cr = om.CaseReader("cases.sql")
    case = cr.get_case(-1)
    values = {key:case.outputs[key] for key in case.outputs if case.outputs[key].shape == (1,)}  # get one-dimensional variables
    df = pd.concat([df, pd.DataFrame(values)], ignore_index=True)  # store in dataframe

In [ ]:
df[['lca:parameters:n_cycles_uav', 'data:weight:mtow', 'lca:aggregation:weighted_single_score:model_per_FU', 'mission:operational:energy', 'data:weight:mtow:requirement', 'data:weight:mtow:requirement:constraint', 'lca:parameters:mass_batteries']]

In [ ]:
df.to_csv('./workdir/df_pareto.csv')

In [ ]:
# Check consistency between energy and lca pareto fronts

# Import dataframes
df_lca = pd.read_csv('./workdir/df_pareto_lca.csv')
df_lca = df_lca.sort_values(by='data:weight:mtow')
df_energy = pd.read_csv('./workdir/df_pareto_energy.csv')
df_energy = df_energy.sort_values(by='data:weight:mtow')

# Create figure with secondary y-axis
fig = go.Figure()

# Add traces
fig.add_trace(
    go.Scatter(x=df_lca['data:weight:mtow'], y=df_lca['lca:aggregation:weighted_single_score:model_per_FU'], name="LCA optim.", 
               line_shape='spline', line_dash='solid', mode='lines+markers'),
)
fig.add_trace(
    go.Scatter(x=df_energy['data:weight:mtow'], y=df_energy['lca:aggregation:weighted_single_score:model_per_FU'], name="Energy optim.", 
               line_shape='spline', line_dash='solid', mode='lines+markers'),
)
fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Import dataframes
#df_lca = pd.read_csv('./workdir/df_pareto_lca.csv')
#df_lca = df_lca.sort_values(by='data:weight:mtow')
#df_energy = pd.read_csv('./workdir/df_pareto_energy.csv')
#df_energy = df_energy.sort_values(by='data:weight:mtow')
# After checking that both dataframe return the same results, we can keep a single one for plotting
#df = df_energy
df = pd.read_csv('./workdir/df_pareto.csv')
df = df.sort_values(by='data:weight:mtow')

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df['data:weight:mtow'], y=df['lca:aggregation:weighted_single_score:model_per_FU'], name="LCA score", 
               line_shape='spline', line_dash='solid', mode='lines', marker_symbol='x', line_width=3.0),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=df['data:weight:mtow'], y=df['mission:operational:energy'], name="Energy consumption", 
               line_shape='spline', line_dash='dashdot', mode='lines', line_width=3.0),
    secondary_y=True,
)

# Min. mass
fig.add_shape(type="line", 
              x0=df['data:weight:mtow'].min(), y0=0.125, 
              x1=df['data:weight:mtow'].min(), y1=df['lca:aggregation:weighted_single_score:model_per_FU'].max(), 
              line_width=1, line_dash="dot", line_color="black")
#fig.add_shape(type="line", 
#              x0=5.35, y0=df['lca:aggregation:weighted_single_score:model_per_FU'].max(), 
#              x1=df['data:weight:mtow'].min(), y1=df['lca:aggregation:weighted_single_score:model_per_FU'].max(), 
#              line_width=1, line_dash="dot", line_color="black")
fig.add_annotation(y=0.14, x=df['data:weight:mtow'].min(), ay=10, text="min. mass", font=dict(color='black'), textangle=-90)

# Min. energy
fig.add_shape(type="line", 
              x0=df.loc[df['mission:operational:energy'].idxmin(), :]['data:weight:mtow'], 
              y0=df['mission:operational:energy'].min(), 
              x1=5.37, 
              y1=df['mission:operational:energy'].min(), 
              line_width=1, line_dash="dot", line_color="red", yref='y2')
fig.add_shape(type="line", 
              x0=df.loc[df['mission:operational:energy'].idxmin(), :]['data:weight:mtow'], 
              y0=620, 
              x1=df.loc[df['mission:operational:energy'].idxmin(), :]['data:weight:mtow'], 
              y1=df['mission:operational:energy'].min(), 
              line_width=1, line_dash="dot", line_color="red", yref='y2')
fig.add_annotation(x=5.28, y=df['mission:operational:energy'].min(), ay=-10, text="min. energy", yref='y2', font=dict(color='red'))

# Min. impact
fig.add_shape(type="line", 
              x0=4.23, 
              y0=df['lca:aggregation:weighted_single_score:model_per_FU'].min(), 
              x1=df.loc[df['lca:aggregation:weighted_single_score:model_per_FU'].idxmin(), :]['data:weight:mtow'], 
              y1=df['lca:aggregation:weighted_single_score:model_per_FU'].min(), 
              line_width=1, line_dash="dot", line_color="blue")
fig.add_shape(type="line", 
              x0=df.loc[df['lca:aggregation:weighted_single_score:model_per_FU'].idxmin(), :]['data:weight:mtow'], 
              y0=0.125, 
              x1=df.loc[df['lca:aggregation:weighted_single_score:model_per_FU'].idxmin(), :]['data:weight:mtow'], 
              y1=df['lca:aggregation:weighted_single_score:model_per_FU'].min(), 
              line_width=1, line_dash="dot", line_color="blue")
fig.add_annotation(x=4.38, y=df['lca:aggregation:weighted_single_score:model_per_FU'].min(), ay=-10, text="min. impact", font=dict(color='blue'))


# Add figure title
fig.update_layout(title=None, width=600, height=400, paper_bgcolor='rgba(0,0,0,0)',  plot_bgcolor='rgba(0,0,0,0)', font=dict(size=14), margin=dict(l=10, r=10, t=5, b=5),
                  legend=dict(x=.5,
                              y=.9,
                              traceorder="normal",
                            )
                 )
fig.update_xaxes(title_text='UAV mass (kg)', showline=True, linewidth=1.0, linecolor='black', showgrid=False, ticks='outside')
fig.update_yaxes(showline=True, linewidth=1, showgrid=False, ticks='outside')
#fig.update_yaxes(range=[0.07666, 0.08658], secondary_y=False)
#fig.update_yaxes(range=[790, 945], secondary_y=True)
#fig.update_xaxes(range=[5.555, 6.35])
fig.update_yaxes(title_text="LCA single score (points)", secondary_y=False, 
                 titlefont=dict(color='blue'),  tickfont=dict(color='blue'), linecolor='blue',
                )
fig.update_yaxes(title_text="Mission energy consumption (kJ)", secondary_y=True, titlefont=dict(color='red'),  tickfont=dict(color='red'), linecolor='red')

fig.show()
plotly.io.write_image(fig, 'output_file.pdf', format='pdf')

In [ ]:
# DESIGN PARAMETERS VARIATION: propeller diameter, battery mass, ...
df = pd.read_csv('./workdir/df_pareto.csv')
df = df.sort_values(by='data:weight:mtow')

# Normalize w.r.t. mass minimization design
df['data:weight:airframe:mass'] = df['data:weight:airframe:arms:mass'] + df['data:weight:airframe:body:mass']
df_norm = df/df.loc[0,:]

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df['data:weight:mtow'], 
               y=df_norm['data:weight:propulsion:multirotor:battery:mass'], 
               name="Battery mass", line_shape='spline', mode='lines', line_width=3),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=df['data:weight:mtow'], 
               y=df_norm['data:weight:propulsion:multirotor:motor:mass'], 
               name="Motor mass", line_shape='spline', mode='lines', line_dash='dot', line_width=3),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=df['data:weight:mtow'], 
               y=df_norm['data:weight:airframe:mass'], 
               name="Airframe mass", line_shape='spline', mode='lines', line_dash='dashdot', line_width=3),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=df['data:weight:mtow'], 
               y=df_norm['data:propulsion:multirotor:propeller:diameter'], 
               name="Propeller diameter", line_shape='spline', mode='lines', line_dash='dash', line_width=3),
    secondary_y=False,
)

fig.add_shape(type="line", 
              x0=df['data:weight:mtow'].min(), y0=.8, 
              x1=df['data:weight:mtow'].min(), y1=2.0, 
              line_width=1, line_dash="dot", line_color="black")
fig.add_annotation(y=1.0, x=df['data:weight:mtow'].min(), ay=0, text="min. mass", font=dict(color='black'), textangle=-90)
fig.add_shape(type="line", 
              x0=df['data:weight:mtow'].max(), y0=.8, 
              x1=df['data:weight:mtow'].max(), y1=2.0, 
              line_width=1, line_dash="dot", line_color="black")
fig.add_annotation(y=1.0, x=df['data:weight:mtow'].max(), ay=0, text="min. energy", font=dict(color='black'), textangle=-90)
fig.add_shape(type="line", 
              x0=5.983233, y0=.8, 
              x1=5.983233, y1=2.0, 
              line_width=1, line_dash="dot", line_color="black")
fig.add_annotation(y=1.0, x=5.983233, ay=0, text="min. impact", font=dict(color='black'), textangle=-90)

# Layout
fig.update_layout(title=None, width=600, height=400, paper_bgcolor='rgba(0,0,0,0)',  plot_bgcolor='rgba(0,0,0,0)', font=dict(size=14), margin=dict(l=10, r=10, t=5, b=5),
                  legend=dict(
                        orientation="h",
                        yanchor="bottom",
                        y=1.0,
                        xanchor="right",
                        x=0.88,
                      font_size=16
                    )
                 )
                 
fig.update_xaxes(title_text='UAV mass (kg)', showline=True, linewidth=1.0, linecolor='black', showgrid=False, ticks='outside', range=[5.55, 6.3])
fig.update_yaxes(title_text='Design parameter variation (-)', showline=True, linewidth=1, showgrid=True, ticks='outside', linecolor='black', gridcolor='grey', gridwidth=0.05)

# Display and save
fig.show()
plotly.io.write_image(fig, 'output_file.pdf', format='pdf')

In [ ]:
# COMPARISON OF THE DETAILED LCA SCORES OF FOR THREE OPTIMUM (1/2)

OUTPUT_FILE_MASS = pth.join(WORK_FOLDER_PATH, "problem_outputs_lca_optim_mass.xml")
OUTPUT_FILE_ENERGY = pth.join(WORK_FOLDER_PATH, "problem_outputs_lca_optim_energy.xml")
OUTPUT_FILE_LCA = pth.join(WORK_FOLDER_PATH, "problem_outputs_lca_optim_lca.xml")

# FINAL SCORE
fig = lca_plot(OUTPUT_FILE_LCA, result_step = 'aggregation', filter_option = 'default', filter_level = 1)
fig.update_layout(
    title=None,
    margin=dict(l=10, r=10, t=0, b=0),
    width=350,
    height=280
)
colors = [px.colors.qualitative.Plotly[0], px.colors.qualitative.Plotly[9]]
fig.update_traces(marker=dict(colors=colors))
fig.show()
plotly.io.write_image(fig, 'output_file.pdf', format='pdf')

In [ ]:
# COMPARISON OF THE DETAILED LCA SCORES OF FOR THREE OPTIMUM (2/2)

output_files_dict = {'mass minimization': OUTPUT_FILE_MASS,
                     'energy minimization': OUTPUT_FILE_ENERGY,
                     'LCA score minimization': OUTPUT_FILE_LCA
                    }
colors = [px.colors.qualitative.Set1[8], px.colors.qualitative.Plotly[1], px.colors.qualitative.Plotly[0]]

fig = go.Figure()

fig_data_ref = lca_plot(OUTPUT_FILE_MASS, result_step = 'characterization', filter_option = 'default', filter_level = 0, percent=False).data[0]

idx = 0
for name, output_file in output_files_dict.items():
    fig_data = lca_plot(output_file, result_step = 'characterization', filter_option = 'default', filter_level = 0, percent=False).data[0]
    #fig.add_trace(go.Bar(x=fig_data.labels, y=fig_data.values))
    fig.add_trace(go.Bar(x=fig_data.x, y=np.asarray(fig_data.y)/np.asarray(fig_data_ref.y), name=name, marker_color=colors[idx]))
    idx += 1

fig.update_layout(barmode='group')
fig.update_layout(title=None, width=850, height=650, paper_bgcolor='rgba(0,0,0,0)',  plot_bgcolor='rgba(0,0,0,0)', font=dict(size=14), legend_title_text='UAV Design',
                 margin=dict(l=10, r=10, t=0, b=0))
fig.update_xaxes(showline=True, linewidth=0.5, linecolor='black', tickangle=90)
fig.update_yaxes(title='Relative score', showline=True, linewidth=1, linecolor='black', gridcolor='grey', gridwidth=0.05)

for idx in range(len(fig.data)):
    fig.data[idx].x = [s.split("<br>", 2)[1] for s in fig.data[idx].x]
    fig.data[idx].marker.line.width = 0

#for idx in range(len(fig.data)):
#    fig.data[idx].x = [s.split("<br>")[1] + " " + s.split("<br>")[3] for s in fig.data[idx].x]
#    fig.data[idx].marker.line.width = 0
    
fig.show()
plotly.io.write_image(fig, 'output_file.pdf', format='pdf')

#### 5 - Batteries comparisons

In [ ]:
# COMPARISON OF THE LCA SCORES

OUTPUT_FILE_NMC = pth.join(WORK_FOLDER_PATH, "problem_outputs_lca_nmc811.xml")
OUTPUT_FILE_LFP = pth.join(WORK_FOLDER_PATH, "problem_outputs_lca_lfp.xml")
OUTPUT_FILE_SI_NMC = pth.join(WORK_FOLDER_PATH, "problem_outputs_lca_si_nmc.xml")

output_files_dict = {'G/NMC': OUTPUT_FILE_NMC,
                     'G/LFP': OUTPUT_FILE_LFP,
                     'Si/NMC': OUTPUT_FILE_SI_NMC
                    }
colors = [px.colors.qualitative.Set1[8], px.colors.qualitative.Plotly[1], px.colors.qualitative.Plotly[0]]

fig = go.Figure()

idx = 0
for name, output_file in output_files_dict.items():
    fig_data = lca_plot(output_file, result_step = 'weighting', filter_option = 'default', filter_level = 0, percent=False).data[0]
    #fig.add_trace(go.Bar(x=fig_data.labels, y=fig_data.values))
    fig.add_trace(go.Bar(x=fig_data.x, y=fig_data.y, name=name, marker_color=colors[idx]))
    idx += 1

fig.update_layout(barmode='group')
fig.update_layout(title=None, width=850, height=650, paper_bgcolor='rgba(0,0,0,0)',  plot_bgcolor='rgba(0,0,0,0)', font=dict(size=14), legend_title_text='Battery chemistry',
                 margin=dict(l=10, r=10, t=0, b=0))
fig.update_xaxes(showline=True, linewidth=0.5, linecolor='black', tickangle=90)
fig.update_yaxes(title='Points', showline=True, linewidth=1, linecolor='black', gridcolor='grey', gridwidth=0.05)

for idx in range(len(fig.data)):
    fig.data[idx].x = [s.split("<br>", 2)[1] for s in fig.data[idx].x]
    fig.data[idx].marker.line.width = 0

#for idx in range(len(fig.data)):
#    fig.data[idx].x = [s.split("<br>")[1] + " " + s.split("<br>")[3] for s in fig.data[idx].x]
#    fig.data[idx].marker.line.width = 0
    
fig.show()
plotly.io.write_image(fig, 'output_file.pdf', format='pdf')

In [ ]:
# COMPARISON OF THE LCA SCORES - NORMALIZED W.R.T BASELINE

OUTPUT_FILE_NMC = pth.join(WORK_FOLDER_PATH, "problem_outputs_lca_nmc811.xml")
OUTPUT_FILE_LFP = pth.join(WORK_FOLDER_PATH, "problem_outputs_lca_lfp.xml")
OUTPUT_FILE_SI_NMC = pth.join(WORK_FOLDER_PATH, "problem_outputs_lca_si_nmc.xml")

output_files_dict = {'G/NMC': OUTPUT_FILE_NMC,
                     'G/LFP': OUTPUT_FILE_LFP,
                     'Si/NMC': OUTPUT_FILE_SI_NMC
                    }
colors = [px.colors.qualitative.Set1[8], px.colors.qualitative.Plotly[1], px.colors.qualitative.Plotly[0]]

fig = go.Figure()

fig_data_ref = lca_plot(OUTPUT_FILE_NMC, result_step = 'characterization', filter_option = 'default', filter_level = 0, percent=False).data[0]

idx = 0
for name, output_file in output_files_dict.items():
    fig_data = lca_plot(output_file, result_step = 'characterization', filter_option = 'default', filter_level = 0, percent=False).data[0]
    #fig.add_trace(go.Bar(x=fig_data.labels, y=fig_data.values))
    fig.add_trace(go.Bar(x=fig_data.x, y=np.asarray(fig_data.y)/np.asarray(fig_data_ref.y), name=name, marker_color=colors[idx]))
    idx += 1

fig.update_layout(barmode='group')
fig.update_layout(title=None, width=850, height=650, paper_bgcolor='rgba(0,0,0,0)',  plot_bgcolor='rgba(0,0,0,0)', font=dict(size=14), legend_title_text='Battery chemistry',
                 margin=dict(l=10, r=10, t=0, b=0))
fig.update_xaxes(showline=True, linewidth=0.5, linecolor='black', tickangle=90)
fig.update_yaxes(title='Relative score', showline=True, linewidth=1, linecolor='black', gridcolor='grey', gridwidth=0.05)

for idx in range(len(fig.data)):
    fig.data[idx].x = [s.split("<br>", 2)[1] for s in fig.data[idx].x]
    fig.data[idx].marker.line.width = 0

#for idx in range(len(fig.data)):
#    fig.data[idx].x = [s.split("<br>")[1] + " " + s.split("<br>")[3] for s in fig.data[idx].x]
#    fig.data[idx].marker.line.width = 0
    
fig.show()
plotly.io.write_image(fig, 'output_file.pdf', format='pdf')

In [ ]:
# VARIATION WITH NUMBER OF CYCLES
df_nmc = pd.read_csv("./workdir/df_cycles_single_scores_nmc.csv")
df_lfp = pd.read_csv("./workdir/df_cycles_single_scores_lfp.csv")
df_si_nmc = pd.read_csv("./workdir/df_cycles_single_scores_si_nmc.csv")

df_nmc['total'] = df_nmc['operation'] + df_nmc['batteries'] + df_nmc['propellers'] + df_nmc['controllers'] + df_nmc['airframe'] + df_nmc['motors']
df_lfp['total'] = df_lfp['operation'] + df_lfp['batteries'] + df_lfp['propellers'] + df_lfp['controllers'] + df_lfp['airframe'] + df_lfp['motors']
df_si_nmc['total'] = df_si_nmc['operation'] + df_si_nmc['batteries'] + df_si_nmc['propellers'] + df_si_nmc['controllers'] + df_si_nmc['airframe'] + df_si_nmc['motors']

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_nmc['n_cycles'], y=df_nmc['total'], line=dict(color=px.colors.qualitative.Dark2[7], dash='solid'), name='G/NMC'))
fig.add_trace(go.Scatter(x=df_lfp['n_cycles'], y=df_lfp['total'], line=dict(color=px.colors.qualitative.Plotly[1], dash='solid'), name='G/LFP'))
fig.add_trace(go.Scatter(x=df_si_nmc['n_cycles'], y=df_si_nmc['total'], line=dict(color=px.colors.qualitative.Plotly[0], dash='solid'), name='Si/NMC'))

fig.update_layout(title=None, width=600, height=300, paper_bgcolor='rgba(0,0,0,0)',  plot_bgcolor='rgba(0,0,0,0)', font=dict(size=14), 
                  legend_title_text='Battery chemistry', legend_traceorder="reversed", margin=dict(l=10, r=10, t=0, b=0))
fig.update_xaxes(title='Number of cycles (-)', showline=True, linewidth=1, linecolor='black', gridcolor='lightgrey', showgrid=True, gridwidth=0.01, tickformat=".f", range=[0,2600])
fig.update_yaxes(title='Single score (points)', showline=True, linewidth=1, linecolor='black', gridcolor='lightgrey', showgrid=True, gridwidth=0.01, range=[0,0.17])
#fig.for_each_trace(lambda trace: trace.update(fillcolor = trace.line.color, marker_line_color = 'black', marker_line_width = 1.0))
fig.show()
plotly.io.write_image(fig, 'output_file.pdf', format='pdf')

#### 6 - Sensitivity to design specifications

In [ ]:
from fastuav.utils.postprocessing.sensitivity_analysis.sensitivity_analysis import doe_fast

# Fresh start
oad.generate_configuration_file(
    CONFIGURATION_FILE, overwrite=True, distribution_name="fastuav", sample_file_name="multirotor_mdo_lca.yaml"
)
oad.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

In [ ]:
# TODO: replace the following code by the "FullFactorialGenerator" driver of OpenMDAO...

import itertools
x_dict = {
    "mission:operational:route_1:payload:mass": np.linspace(2, 5, 10),
    "mission:operational:route_1:cruise:distance": np.linspace(3000, 10000, 10),
    #"mission:operational:route_1:cruise:speed": np.linspace(10, 18, 2)
}
keys, values = zip(*x_dict.items())
permutations_dicts = [dict(zip(keys, v)) for v in itertools.product(*values)]
case_list = [[(key, val) for key, val in permut_dict.items()] for permut_dict in permutations_dicts]
for e in case_list:
    e.append(('mission:sizing:payload:mass', e[0][1]))
    e.append(('mission:operational:route_2:cruise:distance', e[1][1]))
    #e.append(('mission:operational:route_2:cruise:speed', e[2][1])) 
    x_dict['mission:sizing:payload:mass'] = x_dict['mission:operational:route_1:payload:mass']
    x_dict['mission:operational:route_2:cruise:distance'] = x_dict['mission:operational:route_1:cruise:distance']
    #x_dict['mission:operational:route_2:cruise:speed'] = x_dict['mission:operational:route_1:cruise:speed']
len(case_list)

In [ ]:
y_list = ['data:weight:mtow', 'lca:aggregation:weighted_single_score:model_per_FU', 
          'mission:operational:energy', 'data:weight:airframe:arms:mass', 
          'data:weight:airframe:body:mass', 'data:weight:propulsion:multirotor:battery:mass', 
          'data:weight:propulsion:multirotor:esc:mass', 'data:weight:propulsion:multirotor:motor:mass', 
          'data:weight:propulsion:multirotor:propeller:mass', 'data:geometry:arms:number', 'data:geometry:arms:prop_per_arm']

In [ ]:
driver = om.DOEDriver(
            om.ListGenerator(
                data=case_list
            )
        )
df = doe_fast("custom", x_dict, y_list, CONFIGURATION_FILE, custom_driver=driver)

In [ ]:
# CONTOUR PLOT
df = pd.read_csv('./workdir/sensitivity_analysis/doe_custom.csv')
df['mission:operational:route_1:cruise:distance'] /= 1000  # set distance to km
n_cycles = 2600
df['normalized_fu'] = df['lca:aggregation:weighted_single_score:model_per_FU'] / (n_cycles * df['mission:operational:route_1:payload:mass'] * df['mission:operational:route_1:cruise:distance'])
df['operating empty weight'] = df['data:weight:mtow'] - df['mission:operational:route_1:payload:mass']
df = df.drop(df[df.optim_failed == 1.0].index)  # drop failed optim points

In [ ]:
# Set variables to plot
x_name = 'mission:operational:route_1:payload:mass'
y_name = 'mission:operational:route_1:cruise:distance'
z_name = 'normalized_fu'

# Reshape data into a grid
x_vals = sorted(df[x_name].unique())
y_vals = sorted(df[y_name].unique())
z_grid = df.pivot(index=y_name, columns=x_name, values=z_name).values

# Create the contour plot    
fig = go.Figure()
fig.add_trace(go.Contour(
    x=x_vals, y=y_vals, z=z_grid, 
    connectgaps=True,
    contours=dict(start=1.75e-6, end=6.75e-6, size=0.00000025, showlabels=True),
    #contours=dict(start=df[z_name].min(), end=df[z_name].max(), size=0.00000025, showlabels=True),
    #contours_coloring='heatmap' # can also be 'lines', or 'none'
    colorbar=dict(
        title='Single score (points/kg.km)', # title here
        titleside='right',
        titlefont=dict(size=16),
        ticks='outside'
    ),
    colorscale='RdBu_r' # Blackbody,Bluered,Blues,Cividis,Earth,Electric,Greens,Greys,Hot,Jet,Picnic,Portland,Rainbow,RdBu,Reds,Viridis,YlGnBu,YlOrRd. (add '_r' for reversed)
))

#fig.add_trace(go.Scatter(x=df[x_name], y=df[y_name], mode='markers'))

# Update layout
fig.update_layout(title=None, width=600, height=400, paper_bgcolor='rgba(0,0,0,0)',  plot_bgcolor='rgba(0,0,0,0)', font=dict(size=14),
                 margin=dict(l=10, r=10, t=0, b=0))
fig.update_xaxes(title='Payload mass (kg)', ticks='outside', dtick=0.5)
fig.update_yaxes(title='Delivery distance (km)', ticks='outside')

# Display the plot
fig.show()
plotly.io.write_image(fig, 'output_file.pdf', format='pdf')

In [ ]:
# Set variables to plot
x_name = 'mission:operational:route_1:payload:mass'
y_name = 'mission:operational:route_1:cruise:distance'
z_name = 'data:weight:mtow'

# Reshape data into a grid
x_vals = sorted(df[x_name].unique())
y_vals = sorted(df[y_name].unique())
z_grid = df.pivot(index=y_name, columns=x_name, values=z_name).values

# Create the contour plot    
fig = go.Figure()
fig.add_trace(go.Contour(
    x=x_vals, y=y_vals, z=z_grid, 
    connectgaps=True,
    contours=dict(start=2.5, end=18.5, size=1.0, showlabels=True),
    #contours=dict(start=df[z_name].min(), end=df[z_name].max(), size=0.5),
    #contours_coloring='heatmap' # can also be 'lines', or 'none'
    colorbar=dict(
        title='UAV total mass, including payload (kg)', # title here
        titleside='right',
        titlefont=dict(size=16),
        ticks='outside'
    ),
    colorscale='RdBu_r' # Blackbody,Bluered,Blues,Cividis,Earth,Electric,Greens,Greys,Hot,Jet,Picnic,Portland,Rainbow,RdBu,Reds,Viridis,YlGnBu,YlOrRd. (add '_r' for reversed)
))

#fig.add_trace(go.Scatter(x=df[x_name], y=df[y_name], mode='markers'))

# Update layout
fig.update_layout(title=None, width=600, height=400, paper_bgcolor='rgba(0,0,0,0)',  plot_bgcolor='rgba(0,0,0,0)', font=dict(size=14),
                 margin=dict(l=10, r=10, t=0, b=0))
fig.update_xaxes(title='Payload mass (kg)', ticks='outside', dtick=0.5)
fig.update_yaxes(title='Delivery distance (km)', ticks='outside')

# Display the plot
fig.show()
plotly.io.write_image(fig, 'output_file.pdf', format='pdf')

In [ ]:
# Set variables to plot
x_name = 'mission:operational:route_1:payload:mass'
y_name = 'mission:operational:route_1:cruise:distance'
z_name = 'operating empty weight'

# Reshape data into a grid
x_vals = sorted(df[x_name].unique())
y_vals = sorted(df[y_name].unique())
z_grid = df.pivot(index=y_name, columns=x_name, values=z_name).values

# Create the contour plot    
fig = go.Figure()
fig.add_trace(go.Contour(
    x=x_vals, y=y_vals, z=z_grid, 
    connectgaps=True,
    contours=dict(start=2.0, end=12.5, size=0.8, showlabels=True),
    #contours=dict(start=df[z_name].min(), end=df[z_name].max(), size=0.5),
    #contours_coloring='heatmap' # can also be 'lines', or 'none'
    colorbar=dict(
        title='UAV mass, excluding payload (kg)', # title here
        titleside='right',
        titlefont=dict(size=16),
        ticks='outside'
    ),
    colorscale='RdBu_r' # Blackbody,Bluered,Blues,Cividis,Earth,Electric,Greens,Greys,Hot,Jet,Picnic,Portland,Rainbow,RdBu,Reds,Viridis,YlGnBu,YlOrRd. (add '_r' for reversed)
))

#fig.add_trace(go.Scatter(x=df[x_name], y=df[y_name], mode='markers'))

# Update layout
fig.update_layout(title=None, width=600, height=400, paper_bgcolor='rgba(0,0,0,0)',  plot_bgcolor='rgba(0,0,0,0)', font=dict(size=14),
                 margin=dict(l=10, r=10, t=0, b=0))
fig.update_xaxes(title='Payload mass (kg)', ticks='outside', dtick=0.5)
fig.update_yaxes(title='Delivery distance (km)', ticks='outside')

# Display the plot
fig.show()
plotly.io.write_image(fig, 'output_file.pdf', format='pdf')

#### 7 - Assessment against Planetary Boundaries

In [ ]:
def lca_planetary_boundaries(file_path: str, functional_unit_realizations=None, allocation=None, file_formatter=None):
    """
    Returns a plot of the life cycle results compared to planetary boundaries.
    
    :param file_path: xml file containing the lca results (typically expressed per kg.km)
    :param: functional_unit_realizations: number of functional units (typically, number of kg.km) to assess against planetary boundaries.
    :return fig: figure of the life cycle results compared to planetary boundaries, for each category.
    """
    
    if not isinstance(functional_unit_realizations, (list, tuple, np.ndarray)):
        functional_unit_realizations = [functional_unit_realizations]

    # file containing variables and their values
    variables = VariableIO(file_path, file_formatter).read()

    # identifier for lca top-level model
    model_key = LCA_MODEL_KEY.replace(" ", "_")

    # Results to plot : normalized against PBs
    RESULTS_KEY = LCA_NORMALIZATION_KEY

    # look for method names
    methods = {}
    for variable in variables:
        name = variable.name
        if RESULTS_KEY not in name or LCA_FACTOR_KEY in name:
            continue
        method_name = name.split(RESULTS_KEY)[-1].split(":" + model_key)[0]
        unit = variable.description  # units for methods are stored in description column rather than units (not handled by openMDAO units object)
        methods[method_name] = unit

    # Build dataframe of normalized impacts
    df = pd.DataFrame()
    for method, unit in methods.items():
        scores_dict = dict()
        for variable in variables.names():
            if RESULTS_KEY not in variable or method not in variable or LCA_FACTOR_KEY in variable not in variable or model_key not in variable:
                continue
            end = variable.find(":" + model_key)
            full_name = variable[end + 1:] 
            value = variables[variable].value[0]
            scores_dict[full_name] = [value]

        unit_str = f'<br>[{unit}]' if unit != '' else ''
        df2 = pd.DataFrame(scores_dict,
                           index=[method.replace("_", " ").replace(':', '<br>') + unit_str]).transpose()

        df = pd.concat([df, df2], axis=1, ignore_index=False)

    # plots
    data = []
    x = df.columns.values  # each column correspond to an impact assessment method
    
    # Add polar plots for each functional unit value
    f_prev = 0
    for idx in range(len(functional_unit_realizations)):
        f_current = functional_unit_realizations[idx]
        row = df.loc[df.index == model_key]   # Results only for the main model (not the children activities)
        y = [row[method][0] * (f_current - f_prev) for method in x] # multiply values by the number of times we want to achieve the functional unit (typically, total number of kg.km), and substract data from previous set to avoid cumulative effect on bar plot
        trace=go.Barpolar(
            name=str(f_current),
            r=y,
            theta=x,
            #marker=dict(
            #    color=y,  # Set the "r" values as the color values
            #    colorscale='Viridis'  # Choose a colorscale (e.g., 'Viridis')
            #)
        )
        data.append(trace)
        f_prev = f_current
        
    # Add a dashed circle representing allocation budget
    if allocation is not None:
        circle_theta = np.linspace(0, 2 * np.pi, 100)  # Angular values for the circle
        circle_r = np.full(len(x), allocation)  # Set your desired radius here
        data.append(go.Scatterpolar(name = 'Proposed allocation', r=circle_r, theta=x, mode='lines', line_shape='spline', line_dash='dash',  fill='toself'))#, fillcolor='rgba(0, 0, 255, 0.2)',))
        
    fig = go.Figure(data=data)

    return fig, df

In [ ]:
# Retrieve data file
OUTPUT_FILE_PB = pth.join(WORK_FOLDER_PATH, "problem_outputs_lca_planetary_boundaries.xml")

# Define demand 
# E.g. one delivery per week of a 2 kg payload over 7.5 km (15 km round-trip), during one year = 1560 (3120 round-trip) kg.km 
# NB: check how the functional unit is defined (distance = round trip or half? For now, FAST-UAV computes LCA based on total distance so you have to multiply by 2 the delivery distance)
#functional_unit_realizations = [(2*12)*(2*7.5), (2*52)*(2*7.5), (2*2*52)*(2*7.5)]  # [2 kg/month, 2 kg/week, 2*2kg/week] on 7.5 km (2*7.5 = 15 km round-trip)
functional_unit_realizations = [2*180]  # [2 kg/month] on 7.5 km (2*7.5 = 15 km round-trip)
#functional_unit_realizations = [2*336000]  # 336 000 kg.km = last-mile delivery (< 50km) in France in 2019, per capita.
#functional_unit_realizations = [(500+100)*(2*7.5)]  # per capita.year: 500kg food consumed + 100 kg waste (packages+food). Assumption = it is transported over 7.5km.
#functional_unit_realizations = [(2*12)*(2*7.5), (2*52)*(2*7.5), (500+100)*(2*7.5)]  # [2 kg/week, 600kg food+waste/year] on 7.5 km (2*7.5 = 15 km round-trip)
#functional_unit_realizations = [(2*52)*(2*7.5), (500+100)*(2*7.5)]

# Define allocation budget
allocation = 0.1/100

# Plot barpolar chart
fig, df = lca_planetary_boundaries(OUTPUT_FILE_PB, functional_unit_realizations, allocation=allocation)

for idx in range(len(fig.data)):
    fig.data[idx].theta = [s.split("<br>")[1].replace("- ", "<br>") for s in fig.data[idx].theta]
    if idx != len(fig.data)-1:
        fig.data[idx].name = f'Scenario {idx+1} ({functional_unit_realizations[idx] / 2 :.0f} kg.km)'  # division by two: see comment above on round-trip
        #fig.data[idx].name = f'Last mile grocery scenario ({functional_unit_realizations[idx] / 2 :.0f} kg.km/person/year)'  # division by two: see comment above on round-trip
        fig.data[idx].marker.color=px.colors.qualitative.Plotly[idx]
    else:
        fig.data[idx].fillcolor='rgba(255, 0, 0, 0.3)'
        fig.data[idx].line.color='rgba(255, 0, 0, 1.0)'
    
fig.update_layout(
    polar=dict(
        radialaxis=dict(
            angle=90,
            visible=True,
            tickformat=".1%",
            tickangle=90,
            dtick=0.001,
            gridcolor='lightgrey',
            color='black',
            linecolor='grey',
            tickcolor='black',
            ticks='outside'
        ),
        angularaxis=dict(
            gridcolor='lightgrey',
            showline=True,
            linewidth=1.0,
            linecolor='black'
        ),
        bgcolor='rgba(0,0,0,0)',
    hole=0.1
    ),
    title=None, width=800, height=500,
    paper_bgcolor='rgba(0,0,0,0)',  plot_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=100, r=100, t=50, b=50),
    font=dict(size=14),
    legend=dict(yanchor="bottom", xanchor="right", x=.7, y=-.45, bordercolor='black', borderwidth=1),
)

fig.show()
plotly.io.write_image(fig, 'output_file.pdf', format='pdf')

In [ ]:
# Retrieve data file
OUTPUT_FILE_PB = pth.join(WORK_FOLDER_PATH, "problem_outputs_lca_planetary_boundaries.xml")

# Define demand 
# E.g. one delivery per week of a 2 kg payload over 7.5 km (15 km round-trip), during one year = 1560 (3120 round-trip) kg.km 
# NB: check how the functional unit is defined (distance = round trip or half? For now, FAST-UAV computes LCA based on total distance so you have to multiply by 2 the delivery distance)
#functional_unit_realizations = [(2*12)*(2*7.5), (2*52)*(2*7.5), (2*2*52)*(2*7.5)]  # [2 kg/month, 2 kg/week, 2*2kg/week] on 7.5 km (2*7.5 = 15 km round-trip)
#functional_unit_realizations = [2*180]  # [2 kg/month] on 7.5 km (2*7.5 = 15 km round-trip)
#functional_unit_realizations = [2*336000]  # 336 000 kg.km = last-mile delivery (< 50km) in France in 2019, per capita.
#functional_unit_realizations = [(500+100)*(2*7.5)]  # per capita.year: 500kg food consumed + 100 kg waste (packages+food). Assumption = it is transported over 7.5km.
#functional_unit_realizations = [(2*12)*(2*7.5), (2*52)*(2*7.5), (500+100)*(2*7.5)]  # [2 kg/week, 600kg food+waste/year] on 7.5 km (2*7.5 = 15 km round-trip)
functional_unit_realizations = [(2*52)*(2*7.5), (500+100)*(2*7.5)]

# Define allocation budget
allocation = 0.1/100

# Plot barpolar chart
fig, df = lca_planetary_boundaries(OUTPUT_FILE_PB, functional_unit_realizations, allocation=allocation)

for idx in range(len(fig.data)):
    fig.data[idx].theta = [s.split("<br>")[1].replace("- ", "<br>") for s in fig.data[idx].theta]
    if idx != len(fig.data)-1:
        fig.data[idx].name = f'Scenario {idx+2} ({functional_unit_realizations[idx] / 2 :.0f} kg.km)'  # division by two: see comment above on round-trip
        #fig.data[idx].name = f'Last mile grocery scenario ({functional_unit_realizations[idx] / 2 :.0f} kg.km/person/year)'  # division by two: see comment above on round-trip
        fig.data[idx].marker.color=px.colors.qualitative.Plotly[idx+2]
    else:
        fig.data[idx].fillcolor='rgba(255, 0, 0, 0.3)'
        fig.data[idx].line.color='rgba(255, 0, 0, 1.0)'
    
fig.update_layout(
    polar=dict(
        radialaxis=dict(
            type='log',
            angle=90,
            visible=True,
            tickformat=".2%",
            tickangle=90,
            dtick=1,
            range=[-5, -0.01],
            gridcolor='lightgrey',
            color='black',
            linecolor='grey',
            tickcolor='black',
            ticks='outside'
        ),
        angularaxis=dict(
            gridcolor='lightgrey',
            showline=True,
            linewidth=1.0,
            linecolor='black'
        ),
        bgcolor='rgba(0,0,0,0)',
    hole=0.1
    ),
    title=None, width=800, height=500,
    paper_bgcolor='rgba(0,0,0,0)',  plot_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=100, r=100, t=50, b=50),
    font=dict(size=14),
    legend=dict(yanchor="bottom", xanchor="right", x=.7, y=-.45, bordercolor='black', borderwidth=1),
)

fig.show()
plotly.io.write_image(fig, 'output_file.pdf', format='pdf')

In [ ]:
# Heat map of carbon budget consumption w.r.t technology assumption (emissions per kg.km) and demand (kg.km per person per year)

# emissions per kg.km, in gCO2eq/kg.km
x_vals = np.linspace(0.1, 15.0, 100)

# demand in kg.km
y_vals = np.linspace(0, 5000, 100)

# share carbon budget
z_grid = [[x_vals[i] * y_vals[j] / 985000 for i in range(len(x_vals))] for j in range(len(y_vals))] # total emissions related to carbon budget from PB (985 000 gCO2eq per capita)


# Create the contour plot    
fig = go.Figure()
fig.add_trace(go.Contour(
    x=x_vals, y=y_vals, z=z_grid, 
    connectgaps=True,
    contours=dict(start=0.001, end=0.07, size=0.004, showlabels=True, labelformat=".1%"), 
    contours_coloring='heatmap', # can also be 'lines', or 'none'
    colorbar=dict(
        title='Share of safe operating space<br>for climate change', # title here
        titleside='right',
        titlefont=dict(size=16),
        ticks='outside',
        tickformat=".0%",
        #dtick=0.01
    ),
    colorscale='RdBu_r' # Blackbody,Bluered,Blues,Cividis,Earth,Electric,Greens,Greys,Hot,Jet,Picnic,Portland,Rainbow,RdBu,Reds,Viridis,YlGnBu,YlOrRd. (add '_r' for reversed)
))

# Add point for multirotor UAV under study
fig.add_shape(type="line", 
              x0=12.9, y0=0, 
              x1=12.9, y1=5000, 
              line_width=3, line_dash="dot", line_color="white")
fig.add_annotation(y=5000, x=12.9, ay=-17.0, text="Multirotor UAV<br>(case study)", font=dict(color='black', ), textangle=0)

# Same for lightvehicles (VUL < 2.5t)
fig.add_shape(type="rect", 
              x0=0.240, y0=0, 
              x1=1.840, y1=5000, 
              line_width=2, line_dash="dot", line_color="white", fillcolor="white", opacity=0.5)
fig.add_annotation(y=5000, x=1.7, ay=-17.0, text="Diesel Van<br>2-2.5 tons", font=dict(color='black', ), textangle=0)

# Axes titles
fig.update_xaxes(title='Vehicle carbon intensity (gCO<sub>2eq</sub>/kg.km)', ticks='outside', dtick=1.0, titlefont=dict(size=16), showgrid=False)
fig.update_yaxes(title='Demand (kg.km/person.year)', ticks='outside',  tickformat=".0f", tickvals = [0, 1000, 2000, 3000, 4000, 5000], titlefont=dict(size=16),)
fig.update_layout(title=None, width=650, height=400, paper_bgcolor='rgba(0,0,0,0)',  plot_bgcolor='rgba(0,0,0,0)', font=dict(size=14),
                  margin=dict(l=0, t=0, b=0, r=0))

fig.show()
plotly.io.write_image(fig, 'output_file.pdf', format='pdf')

### Draft

In [ ]:
# Set non-float parameters
parameters['elec_switch_param'] = "eu"

# Modify parameter of interest
parameters['n_cycles'] = list(np.geomspace(1.0, 5000, 10000))

# Choose lcia method
method = [('ReCiPe 2016 v1.03, midpoint (E) no LT', 'climate change no LT', 'global warming potential (GWP1000) no LT')]
method = [('ReCiPe 2016 v1.03, midpoint (E) no LT', 'acidification: terrestrial no LT', 'terrestrial acidification potential (TAP) no LT')]

# activities and sub-activities to evaluate
activities = [operation, production]

# Have a look to NiMH batteries
# Run LCA. The DoE is automatically performed.
data = {'n_cycles': parameters['n_cycles']}
parameters['battery_type'] = "nmc_811"
parameters['n_cycles_battery'] = 1000
for act in activities:
    res = lcalg.multiLCAAlgebric(
        act, # The model 
        method, # Impacts
        
        # Parameters of the model
        **parameters
    )
    data[act.as_dict()['name']] = res.iloc[:, 0].values
df1 = pd.DataFrame(data=data, index=parameters['n_cycles'])
df1['flight hours'] = df1['n_cycles'] * parameters['mission_duration']
df1['operation'] = 0.6 * df1['operation']
df1['model'] = df1['operation'] + df1['production']
df1['functional_value'] = df1['n_cycles'] * parameters['mission_duration'] * parameters['mass_payload']

# Do the same with Li-ion batteries
data = {'n_cycles': parameters['n_cycles']}
parameters['battery_type'] = "lfp"
parameters['n_cycles_battery'] = 3000
for act in activities:
    res = lcalg.multiLCAAlgebric(
        act, # The model 
        method, # Impacts
        
        # Parameters of the model
        **parameters
    )
    data[act.as_dict()['name']] = res.iloc[:, 0].values
df2 = pd.DataFrame(data=data, index=parameters['n_cycles'])
df2['flight hours'] = df2['n_cycles'] * parameters['mission_duration']
df2['model'] = df2['operation'] + df2['production']
df2['functional_value'] = df2['n_cycles'] * parameters['mission_duration'] * parameters['mass_payload']

#df['model'] = df['model'] / df['functional_value']
#df['operation'] = df['operation'] / df['functional_value']
#df['production'] = df['production'] / df['functional_value']

In [ ]:
#df.plot(x='flight hours', y=['model', 'operation', 'production'], grid=True)

ax = df1.plot(x='flight hours', y=['production', 'operation', 'model'])
df2.plot(ax=ax, x='flight hours', y=['production', 'operation', 'model'], style=['--', '--', '--'], grid=True)
plt.show()

#### Monte Carlo

In [ ]:
from fastoad.io import VariableIO
from fastuav.constants import PARAM_VARIABLE_KEY

# Select model
model = get_lca_main_activity()  # top-level model

# Get parameters values from problem outputs
variables = VariableIO(DJI_M600_OUTPUT_FILE).read()
param_names = [p for p in variables.names() if p.startswith(PARAM_VARIABLE_KEY)]
parameters = {}
for p in param_names:
    parameters[p.replace(PARAM_VARIABLE_KEY, "")] = variables[p].value[0]

# Choose method
method = ('ReCiPe 2016 v1.03, midpoint (E) no LT', 'climate change no LT', 'global warming potential (GWP1000) no LT')

# Mixed DoE + Monte Carlo
scores = {}
#for elec in list(["eu", "us", "fr"]):  # DoE
#    parameters["elec_switch_param"] = elec  # Set DoE parameter

parameters["elec_switch_param"] = "fr"
for bat in list(["li_ion", "nimh"]):
    parameters["battery_type"] = bat
    if bat == "li_ion":
        parameters["n_cycles_battery"] = 800
    else:
        parameters["n_cycles_battery"] = 1200
    
    # Run Monte Carlo
    res = LCAMonteCarlo(
        model, # the model
        method, # impacts to assess 

        # Number of Monte Carlo runs
        n_runs=1000, 

        # Parameters of the model
        **parameters
    )
    scores[bat] = res[0]  # score distribution

In [ ]:
# plot distributions
for k, v in scores.items():
    plt.hist(v, bins=50);    
plt.show()

In [ ]:
# TEST electricity mix

# Import dataframes
df1 = pd.read_csv('./workdir/df_pareto_lca_eu.csv')
df1 = df1.sort_values(by='data:weight:mtow')
df2 = pd.read_csv('./workdir/df_pareto_lca_fr.csv')
df2 = df2.sort_values(by='data:weight:mtow')

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df1['data:weight:mtow'], y=df1['lca:aggregation:weighted_single_score:model_per_FU'], name="Europe mix", line_shape='spline', mode='lines'),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=df1['data:weight:mtow'], y=df1['mission:operational:energy'], name="Energy consumption", line_shape='spline', line_dash='dash', mode='lines'),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(x=df2['data:weight:mtow'], y=df2['lca:aggregation:weighted_single_score:model_per_FU'], name="France mix", line_shape='spline', mode='lines'),
    secondary_y=False,
)


fig.add_vline(x=df1[df1['lca:aggregation:weighted_single_score:model_per_FU'] == df1['lca:aggregation:weighted_single_score:model_per_FU'].min()]['data:weight:mtow'].values[0], 
              line_width=1, line_dash="dot", line_color="black", annotation_text="min. mass", annotation_position='top left', annotation_textangle=-90)
fig.add_vline(x=df2[df2['lca:aggregation:weighted_single_score:model_per_FU'] == df2['lca:aggregation:weighted_single_score:model_per_FU'].min()]['data:weight:mtow'].values[0], 
              line_width=1, line_dash="dot", line_color="black", annotation_text="min. mass", annotation_position='top left', annotation_textangle=-90)

#fig.add_hline(y=df_energy['mission:operational:energy'].min(), line_width=1, line_dash="dot", line_color="red", secondary_y=True)
#fig.add_hline(y=df_energy['lca:aggregation:weighted_single_score:model_per_FU'].min(), line_width=1, line_dash="dot", line_color="blue", annotation_text="min. impact", annotation_position='top left', annotation_font_color='blue')
#fig.add_annotation(x=6.25, y=df_energy['mission:operational:energy'].min(), ay=10, text="min. energy", yref='y2', font=dict(color='red'))


# Add figure title
fig.update_layout(title=None, width=600, height=400, paper_bgcolor='rgba(0,0,0,0)',  plot_bgcolor='rgba(0,0,0,0)', font=dict(size=14), margin=dict(l=10, r=10, t=5, b=5),
                  legend=dict(x=.5,
                              y=.9,
                              traceorder="normal",
                            )
                 )
fig.update_xaxes(title_text='UAV mass (kg)', showline=True, linewidth=1.0, linecolor='black', showgrid=False, ticks='outside')
fig.update_yaxes(showline=True, linewidth=1, showgrid=False, ticks='outside')
#fig.update_yaxes(range=[0.07666, 0.08658], secondary_y=False)
#fig.update_yaxes(range=[790, 945], secondary_y=True)
#fig.update_xaxes(range=[5.555, 6.35])
fig.update_yaxes(title_text="LCA single score (points)", secondary_y=False, 
                 titlefont=dict(color='blue'),  tickfont=dict(color='blue'), linecolor='blue',
                )
fig.update_yaxes(title_text="Mission energy consumption (kJ)", secondary_y=True, titlefont=dict(color='red'),  tickfont=dict(color='red'), linecolor='red')

fig.show()
plotly.io.write_image(fig, 'output_file.pdf', format='pdf')

In [ ]:
df = pd.DataFrame(data=data, index=parameters['n_cycles_uav'])
df['flight hours'] = df['n_cycles_uav'] * parameters['mission_duration']
df['functional_value'] = df['n_cycles_uav'] * parameters['mission_duration'] * parameters['mass_payload']
df.to_csv(pth.join(DATA_FOLDER_PATH, 'lca_cycles_si_nmc_fr_endpoint.csv'))

#df['model per FU'] = df['model per FU'] / df['functional_value']
#df['operation'] = df['operation'] / df['functional_value']
#df['production'] = df['production'] / df['functional_value']

ax = df.plot(x='flight hours', y=['model per FU'], grid=True)
plt.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# dict for the dataframes and their names
dfs_NMC = {"NMC - FR": pd.read_csv(pth.join(DATA_FOLDER_PATH, 'lca_cycles_nmc811_fr_endpoint.csv')),
           "NMC - EU": pd.read_csv(pth.join(DATA_FOLDER_PATH, 'lca_cycles_nmc811_eu_endpoint.csv')),
           "NMC - US" : pd.read_csv(pth.join(DATA_FOLDER_PATH, 'lca_cycles_nmc811_us_endpoint.csv')),
          }
dfs_LFP = {"LFP - FR" : pd.read_csv(pth.join(DATA_FOLDER_PATH, 'lca_cycles_lfp_fr_endpoint.csv')),
           "LFP - EU" : pd.read_csv(pth.join(DATA_FOLDER_PATH, 'lca_cycles_lfp_eu_endpoint.csv')),
           "LFP - US" : pd.read_csv(pth.join(DATA_FOLDER_PATH, 'lca_cycles_lfp_us_endpoint.csv')),
          }
dfs_SI_NMC = {"SI NMC - FR" : pd.read_csv(pth.join(DATA_FOLDER_PATH, 'lca_cycles_si_nmc_fr_endpoint.csv')),
              "SI NMC - EU" : pd.read_csv(pth.join(DATA_FOLDER_PATH, 'lca_cycles_si_nmc_eu_endpoint.csv')),
              "SI NMC - US" : pd.read_csv(pth.join(DATA_FOLDER_PATH, 'lca_cycles_si_nmc_us_endpoint.csv')),
          }

# plot the data
fig = go.Figure()

for i in dfs_NMC:
    fig = fig.add_trace(go.Scatter(x = dfs_NMC[i]["flight hours"],
                                   y = dfs_NMC[i]["model per FU"], 
                                   name = i,
                                   line = dict(width=2)))
for i in dfs_LFP:
    fig = fig.add_trace(go.Scatter(x = dfs_LFP[i]["flight hours"],
                                   y = dfs_LFP[i]["model per FU"], 
                                   name = i,
                                   line = dict(width=2, dash='dash')))
    
for i in dfs_SI_NMC:
    fig = fig.add_trace(go.Scatter(x = dfs_SI_NMC[i]["flight hours"],
                                   y = dfs_SI_NMC[i]["model per FU"], 
                                   name = i,
                                   line = dict(width=2, dash='dot')))

fig.update_layout(title='Human Health', #Global Warming Potential',
                   xaxis_title='Lifetime (hours)',
                   yaxis_title='DALY', #kgCO2eq/FU',
                  width=900,
                  height=500,)
fig.show()

In [ ]:
import matplotlib.pyplot as plt

def plot_contour(df, x_name, y_name, z_name, levels: int = None):
    """
    Contour plot from dataframe values.
    """
    # Get data
    x = df[x_name]
    y = df[y_name]
    z = df[z_name]
    
    # Initialize plot
    fig, ax = plt.subplots()

    # Plot contour
    ax.tricontour(x, y, z, levels=levels, linewidths=0.5, colors='k')
    cntr = ax.tricontourf(x, y, z, levels=levels, cmap="RdBu_r")
    fig.colorbar(cntr, ax=ax)
    
    # Data points
    ax.plot(x, y, 'ko', ms=3)
    
    return fig

x_name = 'mission:operational:route_1:payload:mass'
y_name = 'mission:operational:route_1:cruise:distance'
z_name = 'lca:aggregation:weighted_single_score:model_per_FU'

fig = plot_contour(df, x_name, y_name, z_name, levels=10)

In [ ]:
import contextlib
import os
import os.path as pth
from fastuav.utils.drivers.salib_doe_driver import SalibDOEDriver
import fastoad.api as oad
from fastoad.io.variable_io import DataFile
import openmdao.api as om
import pandas as pd
import numpy as np
from ipywidgets import widgets, Layout
import plotly.graph_objects as go
from SALib.analyze import sobol, morris
from typing import List
from plotly.validators.scatter.marker import SymbolValidator
import itertools

SA_PATH = "./workdir/sensitivity_analysis"


def doe_fast(
    method_name: str,
    x_dict: dict,
    y_list: List[str],
    conf_file: str,
    ns: int = 100,
    custom_driver=None,
    calc_second_order: bool = True,
) -> pd.DataFrame:
    """
    DoE function for FAST-UAV problems.
    Various generators are available:
        - List generator that reads cases from a provided list of DOE cases
        - Uniform generator provided by pyDOE2 and included in OpenMDAO
        - Latin Hypercube generator provided by OpenMDAO
        - Generator for Sobol-Saltelli 2002 method provided by SALib
        - Generator for Morris method provided by SALib
    If an optimization problem is declared in the configuration file,
    a nested optimization (sub-problem) is run (e.g. to ensure system optimality and/or consistency at each simulation).

    :param method_name: 'uniform', 'lhs', 'Sobol' or 'Morris'
    :param x_dict: inputs dictionary {input_name: [dist_parameter_1, dist_parameter_2, distribution_type]}
    :param y_list: list of problem outputs to record
    :param conf_file: configuration file for the problem
    :param ns: number of samples (Uniform and Sobol) or trajectories (Morris)
    :param calc_second_order: calculate second order indices (Sobol)

    :return: dataframe of the monte carlo simulation results
    """

    class SubProbComp(om.ExplicitComponent):
        """
        Sub-problem component for nested optimization (e.g., to ensure system consistency).
        """

        def initialize(self):
            self.options.declare("conf")
            self.options.declare("x_list")
            self.options.declare("y_list")

        def setup(self):
            # create a sub-problem to use later in the compute
            # sub_conf = oad.FASTOADProblemConfigurator(conf_file)
            conf = self.options["conf"]
            prob = conf.get_problem(read_inputs=True)  # get conf file (design variables, objective, driver...)

            # UNCOMMENT THESE LINES IF USING CMA-ES Driver for solving sub-problem
            # TODO: automatically detect use of CMA-ES driver
            # driver = prob.driver = CMAESDriver()
            # driver.CMAOptions['tolfunhist'] = 1e-4
            # driver.CMAOptions['popsize'] = 100

            # prob.driver.options['disp'] = False
            p = self._prob = prob
            p.setup()

            # set counter for optimization failure
            self._fail_count = 0

            # define the i/o of the component
            x_list = self._x_list = self.options["x_list"]
            y_list = self._y_list = self.options["y_list"]

            for x in x_list:
                self.add_input(x)

            for y in y_list:
                self.add_output(y)
            self.add_output('optim_failed')

            self.declare_partials("*", "*", method="fd")

        def compute(self, inputs, outputs):
            p = self._prob
            x_list = self._x_list
            y_list = self._y_list

            for x in x_list:
                p[x] = inputs[x]

            with open(os.devnull, "w") as f, contextlib.redirect_stdout(
                f
            ):  # turn off all convergence messages (including failures)
                fail = p.run_driver()

            if fail:
                self._fail_count += 1

            for y in y_list:
                outputs[y] = p[y]
            outputs['optim_failed']=fail

    conf = oad.FASTOADProblemConfigurator(conf_file)
    prob_definition = conf.get_optimization_definition()
    x_list = [x_name for x_name in x_dict.keys()]

    # CASE 1: nested optimization is declared (i.e. optimization problem is defined in configuration file)
    if "objective" in prob_definition.keys():
        nested_optimization = True
        prob = om.Problem()
        prob.model.add_subsystem(
            "sub_prob",
            SubProbComp(
                conf=conf,
                x_list=x_list,
                y_list=y_list,
            ),
            promotes=["*"],
        )

    # CASE 2: simple model without optimization
    else:
        nested_optimization = False
        prob = conf.get_problem(read_inputs=True)

    # Setup driver
    if method_name == "list":
        # add input parameters for DoE
        for x_name, x_value in x_dict.items():
            prob.model.add_design_var(
                x_name, lower=x_value.min(), upper=x_value.max()
            )
        # generate all combinations from values in the dict of parameters
        keys, values = zip(*x_dict.items())
        permutations_dicts = [dict(zip(keys, v)) for v in itertools.product(*values)]
        case_list = [[(key, val) for key, val in permut_dict.items()] for permut_dict in permutations_dicts]
        prob.driver = om.DOEDriver(
            om.ListGenerator(
                data=case_list
            )
        )
    elif method_name in ("uniform", "lhs"):
        # add input parameters for DoE
        for x_name, x_value in x_dict.items():
            prob.model.add_design_var(
                x_name, lower=x_value[0], upper=x_value[1]
            )
        # setup driver
        if method_name == "uniform":
            prob.driver = om.DOEDriver(
                om.UniformGenerator(
                    num_samples=ns
                )
            )
        elif method_name == "lhs":
            prob.driver = om.DOEDriver(
                om.LatinHypercubeGenerator(
                    samples=ns
                )
            )
    elif method_name in ("Sobol", "Morris"):
        # add input parameters for DoE
        dists = []
        for x_name, x_value in x_dict.items():
            prob.model.add_design_var(
                x_name, lower=x_value[0], upper=x_value[1]
            )
            dist = x_value[2]  # add distribution type ('unif' or 'norm')
            dists.append(dist)
        # setup driver
        if method_name == "Sobol":
            prob.driver = SalibDOEDriver(
                sa_method_name=method_name,
                sa_doe_options={"n_samples": ns, "calc_second_order": calc_second_order},
                distributions=dists,
            )
        elif method_name == "Morris":
            # setup driver
            prob.driver = SalibDOEDriver(
                sa_method_name="Morris",
                sa_doe_options={"n_trajs": ns},
                distributions=dists,
            )
    elif method_name == "custom":
        # add input parameters for DoE
        for x_name, x_value in x_dict.items():
            prob.model.add_design_var(
                x_name, lower=x_value.min(), upper=x_value.max()
            )
            # setup driver
        prob.driver=custom_driver

    # Attach recorder to the driver
    if os.path.exists("cases.sql"):
        os.remove("cases.sql")
    prob.driver.add_recorder(om.SqliteRecorder("cases.sql"))
    recorded_variables = x_list + y_list
    if nested_optimization:
        recorded_variables.append("optim_failed")
    prob.driver.recording_options["includes"] = recorded_variables

    # Run problem
    prob.setup()
    prob.run_driver()
    prob.cleanup()

    # Get results from recorded cases
    df = pd.DataFrame()
    cr = om.CaseReader("cases.sql")
    cases = cr.list_cases("driver", out_stream=None)
    for case in cases:
        values = cr.get_case(case).outputs
        # df = df.append(values, ignore_index=True)
        df = pd.concat([df, pd.DataFrame(values)], ignore_index=True)

    # for i in df.columns:
    #     df[i] = df[i].apply(lambda x: x[0])

    # Print number of optimization failures
    fail_count = (
        prob.model.sub_prob._fail_count if nested_optimization else 0
    )  # count number of failures for nested optimization
    if fail_count > 0:
        print("%d out of %d optimizations failed." % (fail_count, len(cases)))

    # save to .csv for future use
    df.to_csv(SA_PATH + "/doe_" + method_name + ".csv")

    return df